## 목표
- sklearn 의 datasets 에서 유방암 데이터를 불러와서 악성, 양성을 판단하는 앙상블 모델을 만들어보자

### 데이터 불러오기, 확인

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

In [2]:
cancer_data = load_breast_cancer()
cancer_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [3]:
cancer_data.data
cancer_data.data.shape

(569, 30)

In [4]:
# 문제 데이터 특성 이름 확인
cancer_data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [5]:
# 답 데이터 확인
cancer_data.target
np.unique(cancer_data.target, return_counts=True)

(array([0, 1]), array([212, 357], dtype=int64))

In [6]:
# 답 데이터의 실제 이름
cancer_data.target_names
# 0 - 악성, 1 - 양성

array(['malignant', 'benign'], dtype='<U9')

In [7]:
print(cancer_data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

### Bagging : Random Forest Model
- 몇 개의 트리를 만들 것인지 선택 : n_estimators
- 단일 결정 트리에서 한 것처럼 max_depth등의 매개변수를 조정하여 사전 가지치기를 할 수 있다.
- 회귀와 분류에 모두 사용 가능

In [8]:
# train , test 분리
# X_train, X_test, y_train, y_test
# 문제 : .data  답 : .target
# 문제와 답은 7.5 : 2.5로 분리
# random_state = 5
X_train, X_test, y_train, y_test = train_test_split(
    cancer_data.data, cancer_data.target, test_size=0.25, random_state=5)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(426, 30) (143, 30) (426,) (143,)


In [9]:
# 모델 불러오기
from sklearn.ensemble import RandomForestClassifier
# 모델 객체 생성
rf_model = RandomForestClassifier(n_estimators=120, random_state=5)
# 모델 학습
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=120, random_state=5)

In [10]:
# 교차검증 -> 정확도가 얼마나 나올지 미리 예측
from sklearn.model_selection import cross_val_score
cross_val_score(rf_model, X_train, y_train, cv=5).mean()

0.94593707250342

In [11]:
# 예측
pre = rf_model.predict(X_test)
# 평가 -> 모델 자체적으로 가지고 있는 score 함수 사용
rf_model.score(X_test, y_test)

0.9790209790209791

### GridSearchCV 를 적용하여 모델 성능 향상시키기
- 모델에게 있어 최적의 하이퍼파라미터 조합을 확인해보자

In [12]:
# rf모델 -> n_estimators, max_depth, min_samples_split, max_leaf_nodes 등
from sklearn.model_selection import GridSearchCV
# 교차검증을 통해 최적의 하이퍼파라미터 조합을 찾아주는 도구
# 1. 튜닝할 매개변수를 정의
params = {'n_estimators' : [60, 80, 100], 'max_depth' : [3, 5, 7],
            'min_samples_split' : [4, 8], 'max_leaf_nodes' : [3, 5, 7]}

# 랜덤 포레스트 모델 객체 생성
model = RandomForestClassifier(random_state=5)

# 그리드 서치 모델 객체 생성        # n_jobs = -1 : 코어를 최대한 활용해라
grid = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)

# 학습
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=5), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7], 'max_leaf_nodes': [3, 5, 7],
                         'min_samples_split': [4, 8],
                         'n_estimators': [60, 80, 100]},
             scoring='accuracy')

In [13]:
# 가장 높은 성능일 때의 모델이 가지는 파라미터 조합 확인
print('최고 성능 : ',grid.best_score_)
print('최고 조합 : ',grid.best_params_)

최고 성능 :  0.9435841313269494
최고 조합 :  {'max_depth': 5, 'max_leaf_nodes': 7, 'min_samples_split': 8, 'n_estimators': 80}


### 모델 완성 및 영향력 높은 특성 확인

In [14]:
# 최정의 하이퍼 파라미터로 rf모델 완성하기
rf_best = RandomForestClassifier(n_estimators=80,
                                max_depth=5,
                                max_leaf_nodes=7,
                                min_samples_split=8,
                                random_state = 5)
# 모델 학습
rf_best.fit(X_train,y_train)
# 모델 특성의 중요도 출력
# rf_best.feature_importances_ -> 데이터 프레임으로 보기 좋게 정렬
rf_fi_df = pd.DataFrame(rf_best.feature_importances_,
                        index=cancer_data.feature_names,
                        columns=['importances'])
rf_fi_df.sort_values(by = 'importances', ascending=False).head()

,importances
worst perimeter,0.176065
mean concave points,0.144673
worst area,0.136685
worst concave points,0.114856
worst radius,0.082151


### Boosting : AdaBoost(Adaptive) Model
- RF모델처럼 의사결정트리모델 기반의 모델이지만 각각의 트리들이 의존적으로 존재한다
- 트리모델이 내부적으로 순차적 학습을 진행
- 분류와 회귀에 모두 사용 가능
- 사전 가지치기를 사용할 수 없다

In [15]:
# ABM 불러오기
from sklearn.ensemble import AdaBoostClassifier
# 모델 객체 생성
ada_model = AdaBoostClassifier(n_estimators=5, learning_rate= 1.0, random_state= 5)
ada_model.fit(X_train,y_train)

pre_ada_tr = ada_model.predict(X_train)
pre_ada_te = ada_model.predict(X_test) # 과대적합 제어가 되지 않는 모델이기 때문에 과대적합 확인

from sklearn.metrics import accuracy_score
train_acc = accuracy_score(y_train, pre_ada_tr)
test_acc = accuracy_score(y_test, pre_ada_te)
print("train 정확도 : ", train_acc)
print("test 정확도 : ", test_acc)

train 정확도 :  0.9647887323943662
test 정확도 :  0.958041958041958


c:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


### GridSearchCV 를 적용하여 모델 성능 향상시키기

In [16]:
params = {'n_estimators' : [60, 80, 100], 'learning_rate' : [0.4, 0.6, 0.8, 1.0]}

# 랜덤 포레스트 모델 객체 생성
model = AdaBoostClassifier(random_state=5)

# 그리드 서치 모델 객체 생성        # n_jobs = -1 : 코어를 최대한 활용해라
grid = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)

# 학습
grid.fit(X_train, y_train)

c:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


GridSearchCV(cv=5, estimator=AdaBoostClassifier(random_state=5), n_jobs=-1,
             param_grid={'learning_rate': [0.4, 0.6, 0.8, 1.0],
                         'n_estimators': [60, 80, 100]},
             scoring='accuracy')

In [17]:
# 가장 높은 성능일 때의 모델이 가지는 파라미터 조합 확인
print('최고 성능 : ',grid.best_score_)
print('최고 조합 : ',grid.best_params_)

최고 성능 :  0.9647332421340629
최고 조합 :  {'learning_rate': 0.6, 'n_estimators': 60}


In [18]:
# 최정의 하이퍼 파라미터로 rf모델 완성하기
ada_best = AdaBoostClassifier( n_estimators= 60, learning_rate= 0.6, random_state=5)
# 모델 학습
ada_best.fit(X_train,y_train)
# 모델 특성의 중요도 출력
# rf_best.feature_importances_ -> 데이터 프레임으로 보기 좋게 정렬
ada_fi_df = pd.DataFrame(ada_best.feature_importances_,
                        index=cancer_data.feature_names,
                        columns=['importances'])
ada_fi_df.sort_values(by = 'importances', ascending=False).head()

c:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


,importances
worst area,0.083333
mean texture,0.083333
worst texture,0.083333
area error,0.083333
mean symmetry,0.066667


### Boosting : GradientBoosting Model
- 여러 개의 결정 트리 모델을 묶어 강력한 모델을 만드는 앙상블 기법
- ABM과 유사
- 강력한 사전 가지치기를 사용할 수 있다
- 회귀와 분류에 모두 사용 가능
- 예측 성능이 높지만, 과대적합에 빠르게 걸릴 수 있고, 시간이 오래걸린다.

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
# 객체 생성
# 모델 객체 생성
gbm_model = GradientBoostingClassifier(n_estimators= 110, learning_rate= 0.05
                                        , max_depth=1, random_state=5)

gbm_model.fit(X_train,y_train)

pre_gbm_tr = gbm_model.predict(X_train)
pre_gbm_te = gbm_model.predict(X_test) # 과대적합 제어가 되지 않는 모델이기 때문에 과대적합 확인

from sklearn.metrics import accuracy_score
train_acc = accuracy_score(y_train, pre_gbm_tr)
test_acc = accuracy_score(y_test, pre_gbm_te)
print("train 정확도 : ", train_acc)
print("test 정확도 : ", test_acc)

train 정확도 :  0.9812206572769953
test 정확도 :  0.9790209790209791


In [20]:
params = {'n_estimators' : [110, 120, 130, 140, 150], 'learning_rate' : [0.05, 0.03, 0.01], 'max_depth' : [1, 2, 3]}

# 랜덤 포레스트 모델 객체 생성
model = GradientBoostingClassifier(random_state=5)

# 그리드 서치 모델 객체 생성        # n_jobs = -1 : 코어를 최대한 활용해라
grid = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)

# 학습
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=5),
             n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.03, 0.01],
                         'max_depth': [1, 2, 3],
                         'n_estimators': [110, 120, 130, 140, 150]},
             scoring='accuracy')

In [21]:
# 가장 높은 성능일 때의 모델이 가지는 파라미터 조합 확인
print('최고 성능 : ',grid.best_score_)
print('최고 조합 : ',grid.best_params_)

최고 성능 :  0.9506155950752394
최고 조합 :  {'learning_rate': 0.05, 'max_depth': 1, 'n_estimators': 110}


In [22]:

# 최정의 하이퍼 파라미터로 rf모델 완성하기
gbm_best = GradientBoostingClassifier(n_estimators= 110, learning_rate= 0.05
                                        , max_depth=1, random_state=5)
# 모델 학습
gbm_best.fit(X_train,y_train)
# 모델 특성의 중요도 출력
# rf_best.feature_importances_ -> 데이터 프레임으로 보기 좋게 정렬
gbm_fi_df = pd.DataFrame(gbm_best.feature_importances_,
                        index=cancer_data.feature_names,
                        columns=['importances'])
gbm_fi_df.sort_values(by = 'importances', ascending=False).head()

,importances
worst concave points,0.326864
worst perimeter,0.316428
mean concave points,0.175244
worst area,0.067195
worst radius,0.053732


In [23]:
params = {
    'loss': ['log_loss', 'exponential'],
    'learning_rate': [0.05, 1.0, 1.5, 2.0],
    'n_estimators': [100, 130, 150, 170, 190],
    'max_depth': [1, 2]
    }

# 랜덤 포레스트 모델 객체 생성
model = GradientBoostingClassifier(random_state=5)

# 그리드 서치 모델 객체 생성        # n_jobs = -1 : 코어를 최대한 활용해라
grid = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)

# 학습
grid.fit(X_train, y_train)
# 가장 높은 성능일 때의 모델이 가지는 파라미터 조합 확인
print('최고 성능 : ',grid.best_score_)
print('최고 조합 : ',grid.best_params_)

최고 성능 :  0.9647606019151846
최고 조합 :  {'learning_rate': 1.0, 'loss': 'log_loss', 'max_depth': 1, 'n_estimators': 190}


In [24]:
# 최정의 하이퍼 파라미터로 rf모델 완성하기
gbm_best = GradientBoostingClassifier(
criterion= 'friedman_mse', learning_rate= 0.05, loss= 'exponential', max_depth= 2, n_estimators= 150
                                        )
# 모델 학습
gbm_best.fit(X_train,y_train)
# 모델 특성의 중요도 출력
# rf_best.feature_importances_ -> 데이터 프레임으로 보기 좋게 정렬
gbm_fi_df = pd.DataFrame(gbm_best.feature_importances_,
                        index=cancer_data.feature_names,
                        columns=['importances'])
gbm_fi_df.sort_values(by = 'importances', ascending=False).head()

,importances
worst concave points,0.398350
mean concave points,0.243309
worst area,0.099515
worst perimeter,0.081186
worst radius,0.044653


### XGBoost 모델
- 결정트리 기반의 앙상블 모델에서 각광받고 있는 모델 중 하나
- 분류에 있어 일반적으로 다른 머신러닝 모델보다 성능이 좋은 모델
- GBM을 기반으로 하지만, early stopping(조기학습종료)를 통해서 과대적합을 방지하는 기능이 추가
- 분류와 회귀에 모두 사용 가능

In [25]:
!pip install xgboost

In [28]:
# xgb 불러오기
from xgboost import XGBClassifier
# 객체 생성
xgb_model = XGBClassifier(random_state = 5)
# 학습
xgb_model.fit(X_train, y_train)
# 예측
xgb_model.predict(X_test)
# 평가 
tr_acc = xgb_model.score(X_train, y_train)
te_acc = xgb_model.score(X_test, y_test)

print('train 정확도 : ', tr_acc)
print('test 정확도 : ', te_acc)

train 정확도 :  1.0
test 정확도 :  0.9790209790209791
